In [18]:
import pytesseract
from PIL import Image

# Load the image
image_path = "/Users/hariprasannaa/Desktop/medicine.png"
image = Image.open(image_path)

# Extract text
text = pytesseract.image_to_string(image)

# Split into lines
lines = text.splitlines()

In [19]:
data = []
notification_times = {}
time_noon = {
    "A": "12:53",
    'M': "10:21",
    "N": "09:50"
}

for line in lines:
    if line != "":
        tab_line = line.split("-")
        data.append(
            (tab_line[0].strip(), 1, time_noon[tab_line[2].strip()], tab_line[1].strip()))
        notification_times[time_noon[tab_line[2].strip(
        )]] = f"Please take {tab_line[1].strip()} {tab_line[0].strip()} now"

In [20]:
import subprocess

# Prepare the query
query_tablets = "INSERT INTO TABLETS (name, user_id, time_taken, dosage) VALUES "

# Format the values to insert (handle multiple rows)
values = ", ".join(
    [f"('{name}', {user_id}, '{time_taken}', {dosage})" for name, user_id, time_taken, dosage in data])

# Combine the query and values
cmd = f'mysql -u root -D bmslocal -e "{query_tablets} {values}"'

# Execute in the terminal
subprocess.run(cmd, shell=True)

CompletedProcess(args='mysql -u root -D bmslocal -e "INSERT INTO TABLETS (name, user_id, time_taken, dosage) VALUES  (\'Dolo\', 1, \'12:53\', 1), (\'Citricin\', 1, \'10:21\', 2)"', returncode=0)

In [21]:
import os
import tempfile
from gtts import gTTS
from playsound import playsound
from pync import Notifier
# Translate text using deep-translator (reliable and synchronous)


def translate_text(text, dest_lang, source_lang):
    translator = GoogleTranslator(source=source_lang, target=dest_lang)
    return translator.translate(text)

# Convert translated text to speech and play it


def read_text(text, dest_lang, source_lang):
    otherlang_text = translate_text(text, dest_lang, source_lang)
    Notifier.notify(otherlang_text, title="Reminder")

    # Create and play a temporary audio file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts = gTTS(text=otherlang_text, lang=dest_lang)
        tts.save(fp.name)
        playsound(fp.name)
        os.unlink(fp.name)

In [22]:
notification_times

{'12:53': 'Please take 1 Dolo now', '10:21': 'Please take 2 Citricin now'}

In [23]:
import time
from deep_translator import GoogleTranslator

while True and len(notification_times.keys()) > 0:
    current_time = time.strftime("%H:%M")  # Get current time
    if current_time in notification_times:

        read_text(notification_times[current_time], "ta", "en")

        del notification_times[current_time]
    time.sleep(30)

KeyboardInterrupt: 